# Probing part 1

Vantage points will probe either the targets themselves (step 2) or other addresses in the same /24 prefix (step 2).

Vantage points are only the anchors.  
As always, targets are the anchors.  

This notebook is an implementation of the million scale method. Check the paper for more information.  
To do after create_datasets.ipynb

In [1]:
import uuid

from logger import logger
from scripts.utils.file_utils import load_json
from scripts.utils.measurement_utils import (
    load_targets, 
    load_vps,
    get_measurement_config,
    save_measurement_config,
    get_target_prefixes,
    ping_prefixes,
    ping_targets,
    retrieve_results,
)
from default import (
    USER_ANCHORS_FILE, 
    USER_HITLIST_FILE, 
    PREFIX_MEASUREMENT_RESULTS,
    TARGET_MEASUREMENT_RESULTS,
)

# will define the number of vps and targets to use
NB_TARGETS = 2
NB_VPS = 4

## Load targets and vps dataset

In [2]:

targets = load_targets(USER_ANCHORS_FILE, nb_target=NB_TARGETS)
vps = load_vps(USER_ANCHORS_FILE, nb_vps=NB_VPS)

# all target prefixes
target_prefixes = get_target_prefixes(targets)

# responsive IP addresses in all /24 prefixes
targets_per_prefix = load_json(USER_HITLIST_FILE)

logger.info(f"nb targets: {len(targets)}")
logger.info(f"nb_vps : {len(vps)}")

2023-09-12 20:04:16::INFO:root:2498612375:: nb targets: 2
2023-09-12 20:04:16::INFO:root:2498612375:: nb_vps : 4


# Step 1: probing each target prefixes

## Probe target prefixes
WARNING : Time consumming section

In [3]:
# measurement configuration for retrieval
measurement_uuid = uuid.uuid4()
prefix_measurement_config = get_measurement_config(measurement_uuid, targets, vps)

save_measurement_config(prefix_measurement_config)
ping_prefixes(
    measurement_uuid=measurement_uuid,
    measurement_config=prefix_measurement_config,
    target_prefixes=target_prefixes,
    targets_per_prefix=targets_per_prefix,
    vps=vps,
)

2023-09-12 20:04:16::INFO:root:measurement_utils:: No cached results available
2023-09-12 20:04:16::INFO:root:measurement_utils:: Starting measurements b40bdc7e-deb7-4736-b8ff-c5de371d47f9 with parameters: dry_run=False; nb_targets=5803825; nb_vps=4.


vps list [6543, 6798, 7113, 7022]
vps list [6543, 6798, 7113, 7022]
vps list [6543, 6798, 7113, 7022]
vps list [6543, 6798, 7113, 7022]
vps list [6543, 6798, 7113, 7022]
vps list [6543, 6798, 7113, 7022]


2023-09-12 20:04:19::INFO:root:ping_and_traceroute_classes:: measurement : b40bdc7e-deb7-4736-b8ff-c5de371d47f9 done


# Step 2: probing each target

## Probe targets
WARNING : Time consumming section

In [4]:
# measurement configuration for retrieval
measurement_uuid = uuid.uuid4()
target_measurement_config = get_measurement_config(measurement_uuid, targets, vps)

save_measurement_config(target_measurement_config)

ping_targets(
    measurement_uuid=measurement_uuid,
    measurement_config=target_measurement_config,
    targets=targets,
    vps=vps,
)

2023-09-12 20:04:19::INFO:root:measurement_utils:: No cached results available
2023-09-12 20:04:19::INFO:root:measurement_utils:: Starting measurements 89c4de92-7db8-4340-ac56-775285bc661b with parameters: dry_run=False; nb_targets=2; nb_vps=4.
2023-09-12 20:04:20::INFO:root:ping_and_traceroute_classes:: measurement : 89c4de92-7db8-4340-ac56-775285bc661b done


## Retrieve prefix results
WARNING : Time consuming section

Note: it might take some time before measurement results are available through RIPE API. If no results are available, retry after a few minutes (or hours, it might really depends on the probe itself).

In [10]:
retrieve_results(prefix_measurement_config, PREFIX_MEASUREMENT_RESULTS)

2023-09-12 20:05:50::INFO:root:measurement_utils:: nb measurements retrieved: 6
2023-09-12 20:05:50::INFO:root:measurement_utils:: target: 45.239.44.1 | number of measurement retrieved: 4
2023-09-12 20:05:50::INFO:root:measurement_utils:: target: 91.200.16.4 | number of measurement retrieved: 4
2023-09-12 20:05:50::INFO:root:measurement_utils:: target: 91.200.16.2 | number of measurement retrieved: 4
2023-09-12 20:05:50::INFO:root:measurement_utils:: target: 45.239.44.2 | number of measurement retrieved: 4
2023-09-12 20:05:50::INFO:root:measurement_utils:: target: 45.239.44.6 | number of measurement retrieved: 4
2023-09-12 20:05:50::INFO:root:measurement_utils:: target: 91.200.16.1 | number of measurement retrieved: 4


## Retrieve traget results
WARNING : Time consumming section

Note: it might take some time before measurement results are available through RIPE API. If no results are available, retry after a few minutes (or hours, it might really depends on the probe itself).

In [12]:
retrieve_results(target_measurement_config, TARGET_MEASUREMENT_RESULTS)

2023-09-12 20:05:54::INFO:root:measurement_utils:: nb measurements retrieved: 2
2023-09-12 20:05:54::INFO:root:measurement_utils:: target: 45.239.44.43 | number of measurement retrieved: 4
2023-09-12 20:05:54::INFO:root:measurement_utils:: target: 91.200.16.69 | number of measurement retrieved: 4
